In [21]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `~/SSD/Code/Julia/tinympc-julia/admm-lqr/Project.toml`


In [41]:
using LinearAlgebra
using PyPlot

    Updating registry at `~/.julia/registries/General`


    Updating git-repo `https://github.com/JuliaRegistries/General`


   Resolving package versions...


    Updating `~/SSD/Code/Julia/tinympc-julia/admm-lqr/Project.toml`
  [d330b81b] 

+ PyPlot v2.11.1
    Updating `~/SSD/Code/Julia/tinympc-julia/admm-lqr/Manifest.toml`
  [3da002f7] + ColorTypes v0.11.4
  [5ae59095] + Colors v0.12.10
  [8f4d0f93] + Conda v1.8.0
  [53c48c17] + FixedPointNumbers v0.8.4
  [682c06a0] + JSON v0.21.4
  [b964fa9f] + LaTeXStrings v1.3.0
  [1914dd2f] + MacroTools v0.5.10
  [69de0a69] + Parsers v2.5.9
  [aea7be01] + PrecompileTools v1.1.1
  [21216c6a] + Preferences v1.4.0
  [438e738f] + PyCall v1.95.1
  [d330b81b] + PyPlot v2.11.1
  [189a3867] + Reexport v1.2.2
  [81def892] + VersionParsing v1.3.0
  [0dad84c5] + ArgTools
  [56f22d72] + Artifacts
  [2a0f44e3] + Base64
  [ade2ca70] + Dates
  [f43a241f] + Downloads
  [b77e0a4c] + InteractiveUtils
  [b27032c2] + LibCURL
  [8f399da3] + Libdl
  [37e2e46d] + LinearAlgebra
  [56ddb016] + Logging
  [d6f4376e] + Markdown
  [a63ad114] + Mmap
  [ca575930] + NetworkOptions
  [de0858da] + Printf
  [9a3f8284] + Random
  [ea8e919c] + SHA
  [9e88b42a] + Serialization
  [6462fe0b] + Sockets
  [2f01184e] + Spars

In [42]:
#2D Double-integrator dynamics
h = 0.05 #20 Hz
A = [I(2) h*I(2); zeros(2,2) I(2)]
B = [0.5*h*h*I(2); h*I(2)];

In [43]:
#Reference state trajectory to track (origin)
t = 0:h:10
N = length(t)
xref = zeros(4,N)

#Reference input trajectory
uref = zeros(2,N-1);

In [44]:
#Cost Function
# J = Σ 0.5*(x-xref)'*Q*(x-xref) + 0.5*(u-uref)*R*(u-uref) + 0.5*xn'*Qn*xn
Qn = 10.0*I(4)
Q = 10.0*I(4)
R = 0.1*I(2)

q = zeros(4,N)
r = zeros(2,N-1)

function cost(x,u)
    J = 0.0
    for k = 1:(N-1)
        J += 0.5*(x[:,k]-xref[:,k])'*Q*(x[:,k]-xref[:,k]) + 0.5*(u[:,k]-uref[:,k])'*R*(u[:,k]-uref[:,k])
    end
    J += 0.5*(x[:,N]-xref[:,N])'*Qn*(x[:,N]-xref[:,N])
    return J
end

cost (generic function with 1 method)

In [45]:
#ADMM Functions
function backward_pass!(A,B,Q,q,R,r,P,p,K,d)
    #This is the standard Riccati backward pass with both linear and quadratic terms (like iLQR)
    for k = (N-1):-1:1
        K[:,:,k] .= (R + B'*P[:,:,k+1]*B)\(B'*P[:,:,k+1]*A)
        d[:,k] .= (R + B'*P[:,:,k+1]*B)\(B'*p[:,k+1] + r[:,k])
    
        P[:,:,k] .= Q + K[:,:,k]'*R*K[:,:,k] + (A-B*K[:,:,k])'*P[:,:,k+1]*(A-B*K[:,:,k])
        p[:,k] .= q[:,k] + (A-B*K[:,:,k])'*(p[:,k+1]-P[:,:,k+1]*B*d[:,k]) + K[:,:,k]'*(R*d[:,k]-r[:,k])
    end
end

function backward_pass_grad!(A,B,q,R,r,P,p,K,d)
    #This is just the linear/gradient term from the backward pass (no cost-to-go Hessian or K calculations)
    for k = (N-1):-1:1
        d[:,k] .= (R + B'*P[:,:,k+1]*B)\(B'*p[:,k+1] + r[:,k])
        p[:,k] .= q[:,k] + (A-B*K[:,:,k])'*(p[:,k+1]-P[:,:,k+1]*B*d[:,k]) + K[:,:,k]'*(R*d[:,k]-r[:,k])
    end
end

function forward_pass!(A,B,K,d,x,u)
    for k = 1:(N-1)
        u[:,k] .= -K[:,:,k]*x[:,k] - d[:,k]
        x[:,k+1] .= A*x[:,k] + B*u[:,k]
    end
end

function update_pri!(A,B,q,R̃,r,P,p,K,d,x,u)
    backward_pass_grad!(A,B,q,R̃,r,P,p,K,d)
    forward_pass!(A,B,K,d,x,u)
end
function update_z!(u,z,y)
    #This function clamps the controls to be within the bounds
    for k = 1:(N-1)
        z[:,k] .= min.(umax, max.(umin, u[:,k]+y[:,k]))
    end
end

function update_y!(u,z,y)
    #This function performs the standard AL multiplier update.
    #Note that we're using the "scaled form" where y = λ/ρ
    for k = 1:(N-1)
        y[:,k] .= y[:,k] + u[:,k] - z[:,k]
    end
end

function update_linear_cost!(z,y,r)
    #This function updates the linear term in the control cost to handle the changing cost term from ADMM
    for k = 1:(N-1)
        r[:,k] .= -ρ*(z[:,k]-y[:,k])
    end
end

cost_update! (generic function with 1 method)

In [46]:
#Control Bounds
umin = -[1.0; 1.0]
umax = [1.0; 1.0];

In [47]:
#Algorithm Setup

x = zeros(4,N)
x[:,1] .= [1.0; 0; 0; 1.0]
u = zeros(2,N-1)

ρ = 1.0
R̃ = R + ρ*I

P = zeros(4,4,N)
P[:,:,N] .= Qn
p = zeros(4,N)
p[:,N] .= q[:,N]

K = zeros(2,4,N-1)
d = zeros(2,N-1)

z = zeros(2,N-1);
wnew = zeros(2,N-1);
y = zeros(2,N-1);

backward_pass!(A,B,Q,q,R̃,r,P,p,K,d)

In [48]:
#Main algorithm loop

forward_pass!(A,B,K,d,x,u)
update_z!(u,z,y)
update_y!(u,z,y)
update_linear_cost!(z,y,r)

iter = 1
primal_residual = 1.0
dual_residual = 1.0
while primal_residual > 0.01 || dual_residual > 0.01
    update_pri!(A,B,q,R̃,r,P,p,K,d,x,u)
    update_aux!(u,wnew,y)
    update_dua!(u,wnew,y)
    update_linear_cost!(wnew,y,r)
    
    primal_residual = maximum(abs.(u-z))
    dual_residual = maximum(abs.(ρ*(wnew-z)))
    
    z .= wnew
    
    iter += 1
end

In [49]:
iter

71

In [50]:
plot(xref[1,:])
plot(x[1,:])

1-element Vector{PyCall.PyObject}:
 PyObject <matplotlib.lines.Line2D object at 0x7f7a24e7ee10>

In [51]:
plot(xref[2,:])
plot(x[2,:])

1-element Vector{PyCall.PyObject}:
 PyObject <matplotlib.lines.Line2D object at 0x7f7a22e138d0>

In [52]:
plot(xref[3,:])
plot(x[3,:])

1-element Vector{PyCall.PyObject}:
 PyObject <matplotlib.lines.Line2D object at 0x7f7a22e13fd0>

In [53]:
plot(xref[4,:])
plot(x[4,:])

1-element Vector{PyCall.PyObject}:
 PyObject <matplotlib.lines.Line2D object at 0x7f7a22e1da90>

In [54]:
plot(uref[1,:])
plot(u[1,:])

1-element Vector{PyCall.PyObject}:
 PyObject <matplotlib.lines.Line2D object at 0x7f7a22e13f90>

In [55]:
plot(uref[2,:])
plot(u[2,:])

1-element Vector{PyCall.PyObject}:
 PyObject <matplotlib.lines.Line2D object at 0x7f7a22e27b10>

In [56]:
plot(y[1,:])
plot(y[2,:])

1-element Vector{PyCall.PyObject}:
 PyObject <matplotlib.lines.Line2D object at 0x7f7a22e343d0>

In [57]:
plot(z[1,:])
plot(z[2,:])

1-element Vector{PyCall.PyObject}:
 PyObject <matplotlib.lines.Line2D object at 0x7f7a22e34c50>